In [3]:
import os
print(os.getcwd())
# change working directory to project root
import platform
if platform.system() is "Windows":
    os.chdir('G:\\sven\\git\\cardio')
else:
    os.chdir('/Users/minority/Code/Git/cardio')
print(os.getcwd())
from src.utils.utils_io import Console_and_file_logger, ensure_dir
from src.utils.file_wrapper.app.Files import Basefile, Dicomfile
from src.utils.file_wrapper.app.IO import FileWalker
from src.utils.myshow import myshow, myshow3d

import logging
Console_and_file_logger('numpy_load', logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

from ipywidgets import interact
from src.utils.myshow import myshow, myshow3d

import re
import SimpleITK as sitk
import numpy as np
#import ipyvolume as ipv

G:\sven\git\cardio
G:\sven\git\cardio


In [4]:
# test of the performance of numpy save and load with 4D, 3D and 2D numpy arrays


filename = "data/processed/all/images/0003-04NEJQUZ-2007-03-13_volume_clean.nrrd"
export_path = 'data/processed/nrrd/export/'
export_sitk = 'data/processed/nrrd/export_sitk/'
slice_export_path = 'data/processed/nrrd/export_slice/'
ensure_dir(export_path)
ensure_dir(export_sitk)

img = sitk.ReadImage(filename)
img_nda = sitk.GetArrayFromImage(img)
print(img_nda.shape)



(25, 16, 256, 256)


In [5]:
%%timeit -n 10
# write a 4d nrrd volume with sitk
out_sitk = os.path.join(export_sitk, 'volume{}.nrrd'.format('_' + str(0)))
sitk.WriteImage(img, out_sitk)

1.26 s ± 290 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit -n 10
# save a 4d npy volume with numpy to disk
out_file = os.path.join(export_path, 'volume{}.npy'.format('_' + str(0)))
np.save(out_file, img_nda)

1.29 s ± 268 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [159]:
%%timeit -n 10
# laod a 4d nrrd volume with sitk from disk
img = sitk.ReadImage(filename)

1000 loops, best of 3: 62.7 ms per loop


In [158]:
%%timeit -n 10
# load a 4d volume with numpy from disk
nda_loaded = np.load(out_file)

1000 loops, best of 3: 657 µs per loop


In [68]:
%%timeit
# save 10 x 4D volumes as npy files to disk
for i in range(10):
    
    out_file = os.path.join(export_path, 'volume{}.npy'.format('_' + str(i)))
    np.save(out_file, img_nda)


1 loop, best of 3: 3.02 s per loop


In [58]:
export_path

'data/processed/nrrd/export/'

In [59]:
os.listdir(export_path)

['volume_0.npy',
 'volume_1.npy',
 'volume_2.npy',
 'volume_3.npy',
 'volume_4.npy',
 'volume_5.npy',
 'volume_6.npy',
 'volume_7.npy',
 'volume_8.npy',
 'volume_9.npy']

In [66]:
# size of the exported volumes
sum([os.path.getsize(os.path.join(export_path, f)) for f in os.listdir(export_path)])

880805120

In [67]:
# additional file infos
os.stat(export_path)

os.stat_result(st_mode=16877, st_ino=2692381, st_dev=16777220, st_nlink=12, st_uid=501, st_gid=20, st_size=408, st_atime=1548431589, st_mtime=1548430907, st_ctime=1548430907)

In [69]:
%%timeit
# load 10 4D volumes, with a size of 88MB each
for i in range(10):
    out_file = os.path.join(export_path, 'volume{}.npy'.format('_' + str(i)))
    nda_loaded = np.load(out_file)


1 loop, best of 3: 1.87 s per loop


In [25]:
print(nda_loaded.shape)

(14, 12, 256, 256)


In [84]:
%%timeit
# write all slices of a 4D volume to disk
for t in range(img_nda.shape[0]):
    for z in range(img_nda.shape[1]):
        out_file = os.path.join(slice_export_path, 'slice{}.npy'.format('_' + str(t) + "_" + str(z)))
        np.save(out_file, img_nda[t,z,:,:])
        

1 loop, best of 3: 700 ms per loop


In [91]:
len(os.listdir(slice_export_path))

169

In [83]:
# get the size of all exported slices in MB
sum([os.path.getsize(os.path.join(slice_export_path, f)) for f in os.listdir(slice_export_path)])/(1024*1024.0)

100 loops, best of 3: 3.26 ms per loop


In [125]:
%%timeit
# get a list of all files with os.listdir
files = [os.path.join(slice_export_path,file) for file in sorted(os.listdir(slice_export_path)) if file != '.DS_Store']

1000 loops, best of 3: 1.22 ms per loop


In [131]:
#%%timeit
# get a list of all files with glob
files = glob.glob(os.path.join(slice_export_path, '*.npy'))
len(files)

168

In [130]:
%%timeit
# read all slices of a 4d volume
slices = []
# load all slices of a 4D volume from disk
for file in files:
        slices.append(np.load(file))

1 loop, best of 3: 206 ms per loop


In [103]:
len(slices)

168

In [133]:
%%timeit
single_nda = np.load(files[0])

1000 loops, best of 3: 889 µs per loop


In [110]:
single_nda.shape

(256, 256)

In [119]:

import glob
len(glob.glob(os.path.join(slice_export_path, '*.npy')))

168